# 1. Batch, Mini Batch & Stochastic Gradient Descent

## 1.1. Batch Gradient Descent

The gradient descent that we've seen so far is the Batch Gradient Descent. In Batch Gradient Descent, all the training data is taken into consideration to take a single step. We take the average of the gradients of all the training examples and then use that mean gradient to update our parameters. 

<img src="figures/batch_gradient_descent.png" alt="batch_gradient_descent" style="width: 600px;"/>

Batch Gradient Descent produces a more stable gradient descent convergence with less oscillations and noisy steps taken towards the global minima of the loss function due to updating the parameters by computing the average of all the training samples rather than the value of a single sample. On the other hand, the entire training set can be too large to process in the memory. In addition, the convergence can be slow.

## 1.2. Stochastic Gradient Descent

In Batch Gradient Descent we were considering all the examples for every step of Gradient Descent. This does not seem an efficient way if the dataset is huge. In Stochastic Gradient Descent (SGD), we consider just one example at a time to take a single step.

<img src="figures/stochastic_gradient_descent.png" alt="stochastic_gradient_descent" style="width: 600px;"/>

Basically, in SGD, we are using the cost gradient of 1 example at each iteration, instead of using the mean of the cost gradient of ALL examples.

Since we are considering just one example at a time the cost will fluctuate over the training examples and it will not necessarily decrease. But in the long run, you will see the cost decreasing with fluctuations. Also because the cost is so fluctuating, it will never reach the minima but it will keep dancing around it.
SGD can be used for larger datasets. It converges faster when the dataset is large as it causes updates to the parameters more frequently.

## 1.3. Mini-Batch Gradient Descent

To tackle the issues presented in the two techniques, a mixture of Batch Gradient Descent and SGD is used. So, neither we use all the dataset all at once nor we use the single example at a time. We use a batch of a fixed number of training examples which is less than the actual dataset and call it a mini-batch. Doing this helps us achieve the advantages of both the former variants we saw. 

Mini-batch gradient descent uses $k$ data points (instead of 1 sample in SGD) at each iteration (the number $k$ is also known as the **batch size**). 

<img src="figures/descent.png" alt="descent" style="width: 600px;"/>

## 1.4. Understanding mini-batch gradient descent

* In **Batch gradient descent** we run the gradient descent on the whole dataset.
* While in **Mini-Batch gradient descent** we run the gradient descent on the mini datasets.
* Mini-Batch algorithm pseudo code:
```python
for t = 1:No_of_batches                         # this is called an epoch
	AL, caches = forward_prop(X{t}, Y{t})
	cost = compute_cost(AL, Y{t})
	grads = backward_prop(AL, caches)
	update_parameters(grads)
```
* The code inside an epoch should be vectorized.
* Mini-batch gradient descent works much faster in the large datasets.
* In mini-batch algorithm, the cost won't go down with each step as it does in batch algorithm. It could contain some ups and downs but generally it has to go down (unlike the batch gradient descent where cost function descreases on each iteration).

<img src="figures/mini_batch_gd.png" alt="mini_batch_gd" style="width: 900px;"/>


* Mini-batch size:
    * `(mini batch size = m)` ==> Batch gradient descent
    * `(mini batch size = 1)` ==> Stochastic gradient descent (SGD)
    * `(mini batch size = between 1 and m)` ==> Mini-batch gradient descent
* Batch gradient descent:
    * too long per iteration (epoch)
* Stochastic gradient descent:
    * too noisy regarding cost minimization (can be reduced by using smaller learning rate)
    * won't ever converge (reach the minimum cost)
    * lose speedup from vectorization
* Mini-batch gradient descent:
    * faster learning:
        * you have the vectorization advantage
        * make progress without waiting to process the entire training set
    * doesn't always exactly converge (oscelates in a very small region, but you can reduce learning rate)
* Guidelines for choosing mini-batch size:
    * If small training set (< 2000 examples) - use batch gradient descent.
    * It has to be a power of 2 (because of the way computer memory is layed out and accessed, sometimes your code runs faster if your mini-batch size is a power of 2): `64, 128, 256, 512, 1024, ...`
    * Make sure that mini-batch fits in CPU/GPU memory.
* Mini-batch size is a `hyperparameter`.

# 2. Exponentially Weighted Average

Exponentially Weighted Average is one of the most important algorithms currently in usage. From financial time series, signal processing to neural networks, it is being used quite extensively. Basically, any data that is in a sequence.
This algorithm has been mostly used to reduce the noisy time-series data. It’s also called “smoothing” the data.
The way we achieve this is by essentially weighing the number of observations and using their average. This is called as Moving Average.

<img src="figures/temp_data.png" alt="temp_data" style="width: 500px;"/>

Let’s say we want to calculate moving average of temperature data in the last N days. Data series about the temperature each day are given in $\theta_1, \theta_2, \dots, \theta_N$. With exponentially weighted average, what we do is we start with 0, and then every day, we’re going to combine it with a weight of a parameter (let’s say $\beta$) times whatever is the value accumulated until now plus $1-\beta$ times that day’s temperature.


$$ V_0 = 0 $$
$$ V_1 = \beta V_0 + (1- \beta) \theta_1 $$
$$ V_2 = \beta V_1 + (1- \beta) \theta_2 $$
$$ \vdots $$
$$ V_N = \beta V_{N-1} + (1- \beta) \theta_N $$

In other words, we can write the equation for exponentially weighted average for a given time $t$ as:

$$ V_t = \beta V_{t-1} + (1- \beta) \theta_t $$

<img src="figures/temp_data_ma.png" alt="temp_data_ma" style="width: 500px;"/>


In the figure above, red line shows the weighted average if β is 0.9 and green line shows the weighted average if $\beta$ is 0.98.
By choosing the $\beta$ value, we control how much weight to give for the last $N$ days in calculating the average. $1/1-\beta$ gives us an estimate of the number of days we're averaging over. With $\beta$ as 0.9, we will average over last 10 days of temperature while with $\beta$ as 0.98, we take into account last 50 days of temperature and hence average over the 50 days of temperature.
By averaging over a larger window, the average adapts slowly, when the temperature changes. This is because a lot of weight is given to previous value and a much smaller weight is given to the new value.

So what is the exponentially weighted average actually doing? By expanding the equations for any time $t$, then we can calculate $V_t$ as follows:

\begin{align*} & V_t = (1- \beta) \theta_t +  (1-\beta)\beta \theta_{t-1} +  (1-\beta)\beta^2 \theta_{t-2} +  (1-\beta)\beta^3 \theta_{t-3} +  (1-\beta) \beta^4 \theta_{t-4} + ... \newline
& V_t = (1- \beta) [ \theta_t + \beta \theta_{t-1} +  \beta^2 \theta_{t-2} + \beta^3 \theta_{t-3} +  \beta^4 \theta_{t-4} + ... ]
\end{align*}

<img style="float: right; margin: 0px 0px 15px 15px;" src="figures/ewa_2.png" width="250" />

A bit of intuition of how this formula is exponential decay. As we see below the value of 1-β is what we multiply with the current value. If we expand the equation, we see that we end up multiplying the current value by 1-β and the previous values of β are exponentially decaying on the curve.


Exponential moving average is a highly efficient way to calculate an average. We don’t need much memory or compute power to calculate this average. It’s also highly suited for Machine learning optimizations and using some form of the exponentially weighted average might be helpful over simple gradient descent in neural nets.

## Exponentially weighted averages with bias correction

The above implementation starts with $V_0 = 0$. This makes the algorithm biased at the beginning of it's implementation. It will then take some time to correct this bias.

<img src="figures/ewa_bias_correction.png" alt="ewa_bias_correction" style="width: 500px;"/>

In order to solve this problem, we will add a bias correction term as follows:

$$ V_t = \frac{\beta V_{t-1} + (1- \beta) \theta_t}{1 - \beta^t} $$




# 3. Gradient Descent with Momentum

* The momentum algorithm almost always works faster than standard gradient descent.
* The simple idea is to calculate the exponentially weighted averages for your gradients and then update your weights with the new values.
* Gradient Descent with Momentum allows for a smoother and faster convergence.

* Pseudo code:

```python
vdW = 0, vdb = 0
on iteration t:
	# can be mini-batch or batch gradient descent
	compute dw, db on current mini-batch                
			
	vdW = beta * vdW + (1 - beta) * dW
	vdb = beta * vdb + (1 - beta) * db
	W = W - learning_rate * vdW
	b = b - learning_rate * vdb
```

* Momentum helps the cost function to go to the minimum point in a more fast and consistent way.
* beta is another hyperparameter. beta = 0.9 is very common and works very well in most cases.
* In practice people don't bother implementing bias correction.

<img src="figures/gradient_descent_momentum.png" alt="gradient_descent_momentum" style="width: 700px;"/>

# 4. RMSprop

* RMSprop stands for Root mean square prop.
* This algorithm speeds up the gradient descent.
* Pseudo code:

```python
sdW = 0, sdb = 0
on iteration t:
	# can be mini-batch or batch gradient descent
	compute dw, db on current mini-batch
	
	sdW = (beta * sdW) + (1 - beta) * dW^2  # squaring is element-wise
	sdb = (beta * sdb) + (1 - beta) * db^2  # squaring is element-wise
	W = W - learning_rate * dW / sqrt(sdW)
	b = B - learning_rate * db / sqrt(sdb)
```

* RMSprop will decrease the gradient change when is gradient is large, and increase the gradient change when the gradient is slow.
* Ensure that sdW is not zero by adding a small value epsilon (e.g. epsilon = 10^-8) to it:
    * ```python 
    W = W - learning_rate * dW / (sqrt(sdW) + epsilon)
    ```
* With RMSprop you can increase your learning rate.


# 5. Adam optimization algorithm

* Adam stands for Adaptive Moment Estimation.
* Adam optimization and RMSprop are among the optimization algorithms that worked very well with a lot of NN architectures.
* Adam optimization simply puts RMSprop and momentum together!
* Pseudo code:

```python
vdW = 0, vdW = 0
sdW = 0, sdb = 0
on iteration t:
	# can be mini-batch or batch gradient descent
	compute dw, db on current mini-batch                
			
	vdW = (beta1 * vdW) + (1 - beta1) * dW     # momentum
	vdb = (beta1 * vdb) + (1 - beta1) * db     # momentum
			
	sdW = (beta2 * sdW) + (1 - beta2) * dW^2   # RMSprop
	sdb = (beta2 * sdb) + (1 - beta2) * db^2   # RMSprop
			
	vdW = vdW / (1 - beta1^t)      # fixing bias
	vdb = vdb / (1 - beta1^t)      # fixing bias
			
	sdW = sdW / (1 - beta2^t)      # fixing bias
	sdb = sdb / (1 - beta2^t)      # fixing bias
					
	W = W - learning_rate * vdW / (sqrt(sdW) + epsilon)
	b = B - learning_rate * vdb / (sqrt(sdb) + epsilon)
```

* Hyperparameters for Adam:
    * Learning rate: needed to be tuned.
    * beta1: parameter of the momentum - 0.9 is recommended by default.
    * beta2: parameter of the RMSprop - 0.999 is recommended by default.
    * epsilon: 10^-8 is recommended by default.

# 6. Learning rate decay

* Learning rate decay is the technique of slowly reducing the learning rate after each iteration.
* As mentioned before, it's possible sometimes that we won't reach the optimum point (converge). But by making the learning rate decay with iterations it will be much closer to it because the steps (and possible oscillations) near the optimum are smaller.
* One technique equations is `learning_rate = (1 / (1 + decay_rate * epoch_num)) * learning_rate_0`
    * `epoch_num` is over all data (not a single mini-batch).
* Other learning rate decay methods (continuous):
    * `learning_rate = (0.95 ^ epoch_num) * learning_rate_0`
    * `learning_rate = (k / sqrt(epoch_num)) * learning_rate_0`
* Some people perform learning rate decay discretely - repeatedly decrease after some number of epochs.
* Some people are making changes to the learning rate manually.
* `decay_rate` is another `hyperparameter`.

# 7. The problem of local optima

* The normal local optima is not likely to appear in a deep neural network because data is usually high dimensional. For a point to be a local optima it has to be a local optima for each of the dimensions which is highly unlikely.
* It's unlikely to get stuck in a bad local optima in high dimensions, it is much more likely to get to the saddle point rather to the local optima, which is not a problem.
* Plateaus can make learning slow:
    * Plateau is a region where the derivative is close to zero for a long time.
    * This is where algorithms like momentum, RMSprop or Adam can help.

# 8. Selecting Optimization Algorithm in TensorFlow

Tensorflow optimization algorithms are available [here](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/). Here's an example of selecting the optimisation algorithm with Tensorflow:

```python
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)
```                                    